# COGS 108 - Final Project 

# Overview

*Fill in your overview here*

# Names

- Susmitha Kalidindi 
- Alyssa Ashmore 
- Shardul Saiya 
- Siyi Wang


# Group Members IDs

- A
- A12833150
- A13964199
- A53233349

# Research Question

- Question 1: Want to study how strong the correlation is between words used in product reviews with the ratings given for that product
- Question 2: Want to study the relationship between specific language that users use in reviews, the value of the  ratings they give and whether they are “fake” or incentivized reviews

## Background and Prior Work

*Fill in your background and prior work here* 

References (include links):
- 1)
- 2)

# Hypothesis


*Fill in your hypotheses here*

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [9]:
## Import Data from CSV file here, SEE Project proposal for link to csv file
import pandas as pd 
dfraw = pd.read_csv("../consumer-reviews-of-amazon-products/1429_1.csv", engine = 'python')
df2 = pd.read_csv("../consumer-reviews-of-amazon-products/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv", engine = 'python')


# Data Cleaning

Describe your data cleaning steps here.

In [23]:
# Get rid of most unwanted rows, only keep 
# Name, Category, Reviews.DoRecommend, Review.numHelpful, Reviews.ratings, Review.text, Review.title 

df1 = dfraw.drop(['asins', 'brand', 'keys', 'manufacturer', 'reviews.sourceURLs', 'reviews.userCity', 
                  'reviews.userProvince', 'reviews.username', 'reviews.dateAdded', 'reviews.dateSeen'], axis=1)

# Now df1 contains:
# id, name, categories, reviews.date, reviews.didPurchase, reviews.doRecommend,
# reviews.id, reviews.numHelpful, reviews.rating, reviews.text, reviews.title

for col in df1:
    print(col)


id
name
categories
reviews.date
reviews.didPurchase
reviews.doRecommend
reviews.id
reviews.numHelpful
reviews.rating
reviews.text
reviews.title


In [40]:
# dictionary to count all unique words in customer reviews
words_dict = {}

# Helper function to count words in a string 
# This function directly modifies the words_dict declared above
def word_counter(review):
    
    # Convert to string
    string = str(review)
    
    # Data Filtering
    string = string.replace('.', ' ')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.replace('\'', ' ')
    string = string.replace('\"', ' ')
    string = string.replace('/', ' ')
    string = string.replace('(', ' ')
    string = string.replace(')', ' ')
    string = string.replace('!', ' ')
    string = string.replace('?', ' ')
    string = string.replace('=', ' ')
    string = string.replace('+', ' ')
    string = string.replace('-', ' ')

    # Convert to lower case and split
    my_string = string.lower().split()
    for item in my_string:
        if item in words_dict:
            words_dict[item] += 1
        else:
            words_dict[item] = 1

In [42]:
for review in df1['reviews.text']:
    word_counter(review)
print(words_dict)

{'this': 35428, 'product': 7172, 'so': 12254, 'far': 2046, 'has': 8020, 'not': 11136, 'disappointed': 452, 'my': 34796, 'children': 920, 'love': 13286, 'to': 67722, 'use': 15392, 'it': 70142, 'and': 67912, 'i': 69038, 'like': 7672, 'the': 88622, 'ability': 666, 'monitor': 84, 'control': 1434, 'what': 4654, 'content': 908, 'they': 6014, 'see': 1138, 'with': 19032, 'ease': 646, 'great': 23720, 'for': 52842, 'beginner': 258, 'or': 6216, 'experienced': 52, 'person': 426, 'bought': 10124, 'as': 12860, 'a': 50088, 'gift': 4062, 'she': 8482, 'loves': 5268, 'inexpensive': 548, 'tablet': 18154, 'him': 892, 'learn': 614, 'on': 16902, 'step': 180, 'up': 7114, 'from': 5268, 'nabi': 44, 'he': 4284, 'was': 13042, 'thrilled': 138, 'how': 2624, 'skype': 64, 'already': 932, 've': 1904, 'had': 5894, 'fire': 9708, 'hd': 946, '8': 1034, 'two': 1508, 'weeks': 428, 'now': 3062, 'is': 41764, 'value': 1072, 'we': 7460, 'are': 7560, 'prime': 2818, 'members': 280, 'that': 15744, 'where': 654, 'shines': 16, 'bei

# Data Analysis & Results

Include cells that describe the steps in your data analysis.

In [22]:
# Make Dictionary with 34,500 unique words as keys, and increment their values 

# Ethics & Privacy

*Fill in your ethics & privacy discussion here*

# Conclusion & Discussion

*Fill in your discussion information here*